<a href="https://colab.research.google.com/github/mdaffaalghiffari23/KomputasiIntelegensia/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Muhammad Daffa Alghiffari

NPM: 2106726075

Look up sentiment analysis models on HuggingFace (here)


Run the model on google colab and try to predict a sentence


Check the model’s accuracy using a synthetic dataset


Implement attention transformer on that model


Check the model accuracy after using attention transformer using the same synthetic dataset and compare



# Import

In [ ]:
!pip install transformers


In [ ]:
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00


# Model

In [ ]:
# Install Hugging Face library

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [ ]:
# Tokenize input sentence
sentence = "my boss always gives me unreasonable tasks"
inputs = tokenizer(sentence, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=-1)

# Interpret result
labels = ["negative", "neutral", "positive"]
predicted_label = labels[torch.argmax(predictions)]
confidence = torch.max(predictions).item()

print(f"Predicted sentiment: {predicted_label} with confidence {confidence:.2f}")


Predicted sentiment: negative with confidence 0.96


data: https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis

In [ ]:
import pandas as pd

# Load your dataset
data = pd.read_csv('data.csv')

# Check the structure of the dataset
data=data[:500]


In [ ]:
data

,Sentence,Sentiment,predicted_label
0,The GeoSolutions technology will leverage Bene...,positive,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,neutral
2,"For the last quarter of 2010 , Componenta 's n...",positive,neutral
3,According to the Finnish-Russian Chamber of Co...,neutral,neutral
4,The Swedish buyout firm has sold its remaining...,neutral,neutral
...,...,...,...
495,"In February 2011 , new wording of Identity Doc...",neutral,neutral
496,"Antti Orkola , president of Kemira GrowHow 's ...",negative,negative
497,You 're not alone .,neutral,positive
498,2009 3 February 2010 - Finland-based steel mak...,negative,neutral


In [ ]:
# Tokenize and predict on the dataset
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=-1)
        predicted_label = torch.argmax(predictions).item()
        return predicted_label

# Predict sentiments for each text in the dataset
data['predicted_label'] = data['Sentence'].apply(predict_sentiment)

# Map numeric predictions to labels (0: negative, 1: neutral, 2: positive)
label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
data['predicted_label'] = data['predicted_label'].map(label_mapping)

# Check results
print(data[['Sentence', 'predicted_label']].head())


                                            Sentence predicted_label
0  The GeoSolutions technology will leverage Bene...        positive
1  $ESI on lows, down $1.50 to $2.50 BK a real po...         neutral
2  For the last quarter of 2010 , Componenta 's n...         neutral
3  According to the Finnish-Russian Chamber of Co...         neutral
4  The Swedish buyout firm has sold its remaining...         neutral


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Ensure the labels in your dataset match the predicted labels format
true_labels = data['Sentiment']
predicted_labels = data['predicted_label']

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Model accuracy: {accuracy:.2f}")
print(classification_report(true_labels, predicted_labels))


Model accuracy: 0.67
              precision    recall  f1-score   support

    negative       0.56      0.52      0.54        79
     neutral       0.67      0.79      0.73       278
    positive       0.73      0.50      0.59       143

    accuracy                           0.67       500
   macro avg       0.65      0.60      0.62       500
weighted avg       0.67      0.67      0.66       500



# attention

In [ ]:
from transformers import RobertaForSequenceClassification
import torch
from torch import nn

class CustomRobertaWithAttention(nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomRobertaWithAttention, self).__init__()
        self.roberta = base_model.roberta
        self.attention = nn.MultiheadAttention(embed_dim=base_model.config.hidden_size, num_heads=8)
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        # Get RoBERTa encoder outputs
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        # Apply attention layer
        attention_output, _ = self.attention(hidden_states, hidden_states, hidden_states)

        # Apply the classifier on the [CLS] token (first token)
        logits = self.classifier(attention_output[:, 0, :])

        return logits


In [ ]:
# Load base model
base_model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

# Initialize custom model with attention
custom_model = CustomRobertaWithAttention(base_model, num_labels=3)


In [ ]:
def predict_sentiment_with_attention(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = custom_model(**inputs)
        predictions = torch.softmax(logits, dim=-1)
        predicted_label = torch.argmax(predictions).item()
        return predicted_label

# Predict sentiments with the attention-enhanced model
data['predicted_label_with_attention'] = data['Sentence'].apply(predict_sentiment_with_attention)


In [ ]:
label_mapping = {0: "positive", 1: "negative", 2: "neutral"}

data['predicted_label_with_attention']=data['predicted_label_with_attention'].map(label_mapping)

In [ ]:
predicted_labels_with_attention = data['predicted_label_with_attention']
accuracy_with_attention = accuracy_score(true_labels, predicted_labels_with_attention)
print(f"Model accuracy with attention transformer: {accuracy_with_attention:.2f}")
print(classification_report(true_labels, predicted_labels_with_attention))


Model accuracy with attention transformer: 0.33
              precision    recall  f1-score   support

    negative       0.13      0.11      0.12        79
     neutral       0.41      0.15      0.22       278
    positive       0.35      0.80      0.48       143

    accuracy                           0.33       500
   macro avg       0.29      0.35      0.27       500
weighted avg       0.35      0.33      0.28       500



# Results

In [ ]:
print(f"Accuracy without attention: {accuracy:.2f}")
print(f"Accuracy with attention transformer: {accuracy_with_attention:.2f}")
print(f"Accuracy improvement: {accuracy_with_attention - accuracy:.2f}")


Accuracy without attention: 0.67
Accuracy with attention transformer: 0.33
Accuracy improvement: -0.34
